In [8]:
# !pip install pyautogen
# !pip install openai

# or pip install -r requirements.txt (from the github repo)



For a personal assistant the type of assistant we want likely depends
on the type of activity we do, someone who works with movies will have a 
set of very different requirements than someone who works with software engineering and so on and so forth.

However, we can always think about a common set of tasks that we usually perform daily that are more or less general, things like:

- File management
- Content processing (reading, summarizing, taking notes)
- Performing low level or higher level research
- Certain forms of writing
- Reading through emails and crafting emails or linkedIn messages etc...

Let's imagine this scenario:

We'll always start with some trigger (message from work, jira issue or whatever input) that should be associated with the execution of a task or multiple tasks.

This task should be broken down into an execution flow, for example:

- Task: 'Lucas I need you to test this functionality from an open source framework X. This is the repo: www.github.com/fake/repo.'
- Execution flow:
    - Go to repo
    - Read the readme and find instalation instructions
    - Execute instalation instructions
    - Test the repo according to specifications (if they exist)
    - Write a report
    - Send that report for analysis and validation (to yourself or to another examiner agent)
    - After validation send the final report to contact X. 

Ok, now that we have this overall general picture. Let's see how we can translate that into useful agents built with AutoGen.

The first thing to keep in mind that such a personal assistant would have to have access to our personal information, so we first need a way to 'teach it' 
about us and the important stuff that it should know to be able to execute the tasks to an acceptable degree satisfaction.

For that let's use autogen's `Teachable` class.

In [9]:
from autogen import config_list_from_json

import autogen

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["gpt-4o"]},
    ),
    "temperature": 0,
}

In [10]:
import autogen
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.capabilities.teachability import Teachability

In [11]:
teachable_agent = ConversableAgent(
    name="teachable_agent",
    llm_config=llm_config
)

In [12]:
teachability = Teachability(
    verbosity=0,
    reset_db=True,
    path_to_db_dir="./tmp/notebook/teachability_db",
    recall_threshold=1.5
)

teachability.add_to_agent(teachable_agent)


CLEARING MEMORY


In [13]:
try:
    from termcolor import colored
except ImportError:

    def colored(x, *args, **kwargs):
        return x

In [14]:
user = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=0,
    code_execution_config={
        "work_dir": "work_dir",
        "use_docker": True,
    },
) 

In [15]:
fact1 = "I do all my code development in a folder named code-stuff located here: './code-stuff'"
user.initiate_chat(teachable_agent, message=fact1, clear_history=False)

user (to teachable_agent):

I do all my code development in a folder named code-stuff located here: './code-stuff'

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
teachable_agent (to user):

Got it! I'll remember that you do all your code development in the folder named `code-stuff` located at `./code-stuff`. How can I assist you with your code development today?

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': "I do all my code development in a folder named code-stuff located here: './code-stuff'", 'role': 'assistant'}, {'content': "Got it! I'll remember that you do all your code development in the folder named `code-stuff` located at `./code-stuff`. How can I assist you with your code development today?", 'role': 'user'}], summary="Got it! I'll remember that you do all your code development in the folder named `code-stuff` located at `./code-stuff`. How can I assist you with your code development today?", cost={'usage_including_cached_inference': {'total_cost': 0.000865, 'gpt-4o-2024-05-13': {'cost': 0.000865, 'prompt_tokens': 53, 'completion_tokens': 40, 'total_tokens': 93}}, 'usage_excluding_cached_inference': {'total_cost': 0.000865, 'gpt-4o-2024-05-13': {'cost': 0.000865, 'prompt_tokens': 53, 'completion_tokens': 40, 'total_tokens': 93}}}, human_input=[])

In [16]:
fact2 = "Currently my active projects are: image-classification for client1; technical-writing for client2; llm-automations for client3."
user.initiate_chat(teachable_agent, message=fact2, clear_history=False)

user (to teachable_agent):

Currently my active projects are: image-classification for client1; technical-writing for client2; llm-automations for client3.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
teachable_agent (to user):

Understood! Your active projects are:

1. **image-classification** for client1
2. **technical-writing** for client2
3. **llm-automations** for client3

And all your code development is done in the `code-stuff` folder located at `./code-stuff`. How can I assist you with these projects today?

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': "I do all my code development in a folder named code-stuff located here: './code-stuff'", 'role': 'assistant'}, {'content': "Got it! I'll remember that you do all your code development in the folder named `code-stuff` located at `./code-stuff`. How can I assist you with your code development today?", 'role': 'user'}, {'content': 'Currently my active projects are: image-classification for client1; technical-writing for client2; llm-automations for client3.', 'role': 'assistant'}, {'content': 'Understood! Your active projects are:\n\n1. **image-classification** for client1\n2. **technical-writing** for client2\n3. **llm-automations** for client3\n\nAnd all your code development is done in the `code-stuff` folder located at `./code-stuff`. How can I assist you with these projects today?', 'role': 'user'}], summary='Understood! Your active projects are:\n\n1. **image-classification** for client1\n2. **technical-writing** for client2\n3. **

In [17]:
fake_request = """Client1 just requested the following: 'Lucas, train a pytorch model for classifying images of cats and dogs. You can use publicly available datasets. I need it by next week.'\
    Create a folder in the proper location and write all the scripts necessary to finish this task. Then, write a short report as report.txt in the proper folder."""
user.initiate_chat(teachable_agent, message=fake_request, clear_history=False)

user (to teachable_agent):

Client1 just requested the following: 'Lucas, train a pytorch model for classifying images of cats and dogs. You can use publicly available datasets. I need it by next week.'    Create a folder in the proper location and write all the scripts necessary to finish this task. Then, write a short report as report.txt in the proper folder.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
teachable_agent (to user):

Sure, I'll guide you through the process of setting up the project, writing the necessary scripts, and creating the report.

### Step 1: Create the Project Folder
First, let's create a folder for the `image-classification` project for client1.

```bash
mkdir -p ./code-stuff/image-classification
```

### Step 2: Set Up the Environment
Create a virtual environment and install the necessary packages.

```bash
cd ./code-stuff/image-classification
python3 -m venv venv
source venv/bin/activate
pi

ChatResult(chat_id=None, chat_history=[{'content': "I do all my code development in a folder named code-stuff located here: './code-stuff'", 'role': 'assistant'}, {'content': "Got it! I'll remember that you do all your code development in the folder named `code-stuff` located at `./code-stuff`. How can I assist you with your code development today?", 'role': 'user'}, {'content': 'Currently my active projects are: image-classification for client1; technical-writing for client2; llm-automations for client3.', 'role': 'assistant'}, {'content': 'Understood! Your active projects are:\n\n1. **image-classification** for client1\n2. **technical-writing** for client2\n3. **llm-automations** for client3\n\nAnd all your code development is done in the `code-stuff` folder located at `./code-stuff`. How can I assist you with these projects today?', 'role': 'user'}, {'content': "Client1 just requested the following: 'Lucas, train a pytorch model for classifying images of cats and dogs. You can use p

As we can see the agent incorporates the information provided during the "teaching" portion of the workflow in order to properly execute the tasks as desired.

In [18]:
fact3 = "I am currently working on a side project about how to leverage large language models to augment human capabilities and problem solving skills."

user.initiate_chat(teachable_agent, message=fact3, clear_history=False)

user (to teachable_agent):

I am currently working on a side project about how to leverage large language models to augment human capabilities and problem solving skills.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
teachable_agent (to user):

Got it! You're working on a side project about leveraging large language models to augment human capabilities and problem-solving skills. I'll keep that in mind along with your active projects and development folder.

Would you like to create a new folder for this side project in your `code-stuff` directory, or do you need help with something specific related to this side project?

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': "I do all my code development in a folder named code-stuff located here: './code-stuff'", 'role': 'assistant'}, {'content': "Got it! I'll remember that you do all your code development in the folder named `code-stuff` located at `./code-stuff`. How can I assist you with your code development today?", 'role': 'user'}, {'content': 'Currently my active projects are: image-classification for client1; technical-writing for client2; llm-automations for client3.', 'role': 'assistant'}, {'content': 'Understood! Your active projects are:\n\n1. **image-classification** for client1\n2. **technical-writing** for client2\n3. **llm-automations** for client3\n\nAnd all your code development is done in the `code-stuff` folder located at `./code-stuff`. How can I assist you with these projects today?', 'role': 'user'}, {'content': "Client1 just requested the following: 'Lucas, train a pytorch model for classifying images of cats and dogs. You can use p

In [19]:
fake_request2 = "Research 10 interesting arxiv papers for me related to the side project I am currently working on."
user.initiate_chat(teachable_agent, message=fake_request2, clear_history=False)

user (to teachable_agent):

Research 10 interesting arxiv papers for me related to the side project I am currently working on.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
teachable_agent (to user):

Sure, I'll find 10 interesting arXiv papers related to leveraging large language models to augment human capabilities and problem-solving skills. I'll also create a folder for this side project and write a short report summarizing the findings.

### Step 1: Create the Project Folder
First, let's create a folder for your side project.

```bash
mkdir -p ./code-stuff/llm-augmentation
```

### Step 2: Research and Summarize Papers
Here are 10 interesting arXiv papers related to your side project:

1. **GPT-3: Language Models are Few-Shot Learners**
   - **Authors:** Tom B. Brown, Benjamin Mann, Nick Ryder, et al.
   - **Summary:** This paper introduces GPT-3, a state-of-the-art language model that demonstrates strong few-shot l

ChatResult(chat_id=None, chat_history=[{'content': "I do all my code development in a folder named code-stuff located here: './code-stuff'", 'role': 'assistant'}, {'content': "Got it! I'll remember that you do all your code development in the folder named `code-stuff` located at `./code-stuff`. How can I assist you with your code development today?", 'role': 'user'}, {'content': 'Currently my active projects are: image-classification for client1; technical-writing for client2; llm-automations for client3.', 'role': 'assistant'}, {'content': 'Understood! Your active projects are:\n\n1. **image-classification** for client1\n2. **technical-writing** for client2\n3. **llm-automations** for client3\n\nAnd all your code development is done in the `code-stuff` folder located at `./code-stuff`. How can I assist you with these projects today?', 'role': 'user'}, {'content': "Client1 just requested the following: 'Lucas, train a pytorch model for classifying images of cats and dogs. You can use p